In [ ]:
%load_ext jupyter_black
import torch
import torch.optim as optim
import torch.nn as nn
from optimizers.lrmsprop import LRMSprop
from optimizers.ladam import LAdam
from optimizers.ladadelta import LAdadelta
from optimizers.lsgd import APLSGD
from optimizers.ladagrad import LAdagrad
import matplotlib.pyplot as plt
import numpy as np
import scienceplots
from training.train import train, plot_langevin_loss
from diffusion.deephedging import DeepHedging
from scheduler import PiecewiseConstantScheduler
import os

plt.style.use(["science", "no-latex", "grid"])

seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In this problem, we attempt to replicate a $\mathcal F_T\text{-measurable}$ payoff $Z$ defined on some portfolio $S_t\in\mathbb R^{d_1}$. The control $u_t\in\mathbb R^{d_1}$ represents the amount held at time $t$ of each asset. The objective is :
$$ J(u, w) := \mathbb E\left[w + l\left(Z - \sum_{k=0}^{N-1} \left\langle u_{t_k}, S_{t_{k+1}} - S_{t_k}\right\rangle + \sum_{k=0}^{N-1} \left\langle c_{t_r}S_{t_k}, \left|u_{t_k} - u_{t_{k-1}}\right|\right\rangle - w \right)\right]$$

Where $l: \mathbb R\to \mathbb R$ is a continuous, non-decreasing convex function. In our experiments we will consider the loss function associated with the value-at-risk i.e: $l: x\mapsto (1-\alpha)^{-1}\max(x, 0)$. We will also work in the case where $d_1$ is even, such that for $d'_1 = d_1 / 2$ the assets $(S_t^{1, i})_{1\leq i \leq d'_1} = (S_t^i)_{1\leq i \leq d'_1}$ follow $d'_1$ independant Heston dynamics, and 
$(S_t^{2, i})_{1\leq i \leq d'_1} = (S_t^i)_{d'_1 +1\leq i \leq d_1}$ are the corresponding variance swaps, i.e:
$$\left\{\begin{align}
    &dS_t^{1, i} = \sqrt{V_t^i} S_t^{1, i} \,dB^i_t\;,\;\; S_0^{1,i} = s_0^i\\
    & dV_t^i = a^i(b^i - V_t^i)dt + \eta^i\sqrt{V_t^i}\,dW^i_t\;,\;\;V_0^i = v_0^i\\
    & d\left\langle B^i, W^i\right\rangle_t = \rho^i
\end{align} \right.$$
and :
$$\begin{align}
    &S_t^{2, i} := \mathbb E\left[\int_0^T V_s^i\,ds\Big | \mathcal F_t\right] = \int_0^t V_s^i\,ds + L^i(t, V_t^i) \\
    & L^i(t, v) := \frac{v - b^i}{a^i}\left(1 - e^{- a^i(T-t)}\right) + b^i(T-t)
\end{align}$$
The payoff is chosen to be call options over the tradable assets $S^1$ i.e: $Z = \sum_{i=1} ^{d'_1} \left(S_T^{1, i} - K^i\right)_+$.
For the parameters:
$$\begin{align}
& d'_1 = 5, \; T=1, \; a=1_{\mathbb R^5}, \;b=0.04 \times 1_{\mathbb R^5}, \; \eta = 2\times 1_{\mathbb R^5}, \; \rho = -0.7 \times 1_{\mathbb R^5}, \; \alpha = 0.9, \\
&s_0 = K = 1_{\mathbb R^5}, \; v_0 = 0.1 \times 1_{\mathbb R^5}, \; c_{t_r} = 5e-4\times 1_{\mathbb R^5}.
\end{align}$$

In [ ]:
train_batch = 512
test_batch = 512
train_size = 5
test_size = 25

### Single control:

In [ ]:
def test_diag_single_ctrl(
    model_config,
    N_euler,
    control_config,
    optim,
    langevin_optim,
    name,
    lr,
    target_lr,
    sigma,
    epochs,
    total_iters,
    **optim_kwargs,
):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))

    for i, N in enumerate(N_euler):
        # Vanilla algo
        ocp = DeepHedging(**model_config, N_euler=N)
        ocp.set_control(control_config, multiple_controls=False)
        optim_w = torch.optim.SGD(ocp.w.parameters(), lr=2e-3, momentum=0.9)
        vanilla_optim = optim(ocp.control.parameters(), lr=lr, **optim_kwargs)
        scheduler = PiecewiseConstantScheduler(
            vanilla_optim, target_lr=target_lr, target_sigma=0, total_iters=total_iters
        )
        scheduler_w = PiecewiseConstantScheduler(
            optim_w, target_lr=target_lr, target_sigma=0, total_iters=total_iters
        )
        train_loss, test_loss, test_ci = train(
            ocp,
            [vanilla_optim, optim_w],
            [scheduler, scheduler_w],
            name,
            epochs,
            train_size,
            test_size,
            train_batch,
            test_batch,
        )
        axs[i].plot(
            np.arange(len(test_loss)),
            test_loss,
            marker="o",
            mec="k",
            ms=3,
            label=name,
        )
        axs[i].fill_between(
            np.arange(len(test_loss)),
            np.array(test_loss) - np.array(test_ci),
            np.array(test_loss) + np.array(test_ci),
            alpha=0.2,
        )

        # Langevin algo
        ocp = DeepHedging(**model_config, N_euler=N)
        ocp.set_control(control_config, multiple_controls=False)
        optim_w = torch.optim.SGD(ocp.w.parameters(), lr=2e-3, momentum=0.9)
        loptim = langevin_optim(ocp.control.parameters(), lr=lr, sigma=sigma[i])
        scheduler = PiecewiseConstantScheduler(
            loptim, target_lr=target_lr, target_sigma=0, total_iters=total_iters
        )
        scheduler_w = PiecewiseConstantScheduler(
            optim_w, target_lr=target_lr, target_sigma=0, total_iters=total_iters
        )
        train_loss, test_loss, test_ci = train(
            ocp,
            [loptim, optim_w],
            [scheduler, scheduler_w],
            f"L-{name}",
            epochs,
            train_size,
            test_size,
            train_batch,
            test_batch,
        )
        axs[i].plot(
            np.arange(len(test_loss)),
            test_loss,
            marker="o",
            mec="k",
            ms=3,
            label=f"L-{name}",
        )
        axs[i].fill_between(
            np.arange(len(test_loss)),
            np.array(test_loss) - np.array(test_ci),
            np.array(test_loss) + np.array(test_ci),
            alpha=0.2,
        )

        if i == 0:
            axs[i].set_ylabel(r"$J(u_\theta)$")
        axs[i].set_xlabel("Epochs")
        if i == 2:
            legend = axs[i].legend(fancybox=True, edgecolor="k", loc=0)
            legend.get_frame().set_linewidth(0.5)

        axs[i].set_ylim((0, 1.2))

    plt.show()
    plt.close()

    return fig

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alpha = 0.9
ell = lambda x: nn.functional.relu(x) / (1.0 - alpha)
dim = 5
s0 = 1.0
v0 = 0.1


model_config = dict(
    T=1.0,
    dim=dim,
    ell=ell,
    a=torch.ones(dim).to(device),
    b=0.04 * torch.ones(dim).to(device),
    sigma=2.0 * torch.ones(dim).to(device),
    rho=-0.7 * torch.ones(dim).to(device),
    K=s0 * torch.ones(dim).to(device),
    T_COST=5e-4,
    S_0=s0 * torch.ones(dim).to(device),
    V_0=v0 * torch.ones(dim).to(device),
    device=device,
)

control_config = dict(
    input_dim=1 + 4 * dim,
    output_dim=2 * dim,
    hidden_dim=32,
    depth=2,
    activation=nn.ReLU(),
    out_transform=nn.ReLU(),
)

In [ ]:
adam_single_ctrl = test_diag_single_ctrl(
    model_config,
    [30, 50, 100],
    control_config,
    torch.optim.Adam,
    LAdam,
    "Adam",
    2e-3,
    2e-4,
    [2e-4, 2e-4, 2e-4],
    100,
    80,
)

In [ ]:
adagrad_single_ctrl = test_diag_single_ctrl(
    model_config,
    [30, 50, 100],
    control_config,
    torch.optim.Adagrad,
    LAdagrad,
    "Adagrad",
    7e-3,
    7e-4,
    [1e-5, 1e-5, 1e-5],
    100,
    80,
)

In [ ]:
adadelta_single_ctrl = test_diag_single_ctrl(
    model_config,
    [30, 50, 100],
    control_config,
    torch.optim.Adadelta,
    LAdadelta,
    "Adadelta",
    5e-1,
    5e-2,
    [5e-3, 5e-3, 5e-3],
    100,
    80,
    rho=0.95,
)

In [ ]:
rmsprop_single_ctrl = test_diag_single_ctrl(
    model_config,
    [30, 50, 100],
    control_config,
    torch.optim.RMSprop,
    LRMSprop,
    "RMSprop",
    2e-3,
    2e-4,
    [2e-4, 2e-4, 2e-4],
    100,
    80,
)

In [ ]:
sgd_single_ctrl = test_diag_single_ctrl(
    model_config,
    [30, 50, 100],
    control_config,
    torch.optim.SGD,
    APLSGD,
    "SGD",
    3e-2,
    3e-3,
    [5e-5, 5e-5, 5e-5],
    100,
    80,
    momentum=0.9,
)

In [ ]:
if os.path.isdir("graphs/deephedging"):
    adam_single_ctrl.savefig("graphs/deephedging/adam_single_ctrl.pdf")
    adadelta_single_ctrl.savefig("graphs/deephedging/adadelta_single_ctrl.pdf")
    rmsprop_single_ctrl.savefig("graphs/deephedging/rmsprop_single_ctrl.pdf")
    sgd_single_ctrl.savefig("graphs/deephedging/sgd_single_ctrl.pdf")
    adagrad_single_ctrl.savefig("graphs/deephedging/adagrad_single_ctrl.pdf")

### Multiple controls:

In [ ]:
def test_diag_multiple_ctrls(
    model_config,
    N_euler,
    control_config,
    optim,
    langevin_optim,
    name,
    lr,
    target_lr,
    sigma,
    ll,
    epochs,
    total_iters,
    **optim_kwargs,
):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))

    for i, N in enumerate(N_euler):
        # Vanilla algo
        ocp = DeepHedging(**model_config, N_euler=N)
        ocp.set_control(control_config, multiple_controls=True)
        optim_w = torch.optim.SGD(ocp.w.parameters(), lr=2e-3, momentum=0.9)
        vanilla_optim = [
            optim(control.parameters(), lr=lr, **optim_kwargs)
            for control in ocp.control
        ]
        scheduler_w = PiecewiseConstantScheduler(
            optim_w, target_lr=target_lr, target_sigma=0, total_iters=total_iters
        )
        scheduler = [
            PiecewiseConstantScheduler(
                optimizer, target_lr=target_lr, target_sigma=0, total_iters=total_iters
            )
            for optimizer in vanilla_optim
        ]
        train_loss, test_loss, test_ci = train(
            ocp,
            [optim_w, *vanilla_optim],
            [scheduler_w, *scheduler],
            name,
            epochs,
            train_size,
            test_size,
            train_batch,
            test_batch,
        )
        axs[i].plot(np.arange(len(test_loss)), test_loss, label=name)
        axs[i].fill_between(
            np.arange(len(test_loss)),
            np.array(test_loss) - np.array(test_ci),
            np.array(test_loss) + np.array(test_ci),
            alpha=0.2,
        )

        # Langevin algo
        ocp = DeepHedging(**model_config, N_euler=N)
        ocp.set_control(control_config, multiple_controls=True)
        optim_w = torch.optim.SGD(ocp.w.parameters(), lr=2e-3, momentum=0.9)
        loptim = [
            langevin_optim(control.parameters(), lr=lr, sigma=sigma[i])
            for control in ocp.control
        ]
        scheduler_w = PiecewiseConstantScheduler(
            optim_w, target_lr=target_lr, target_sigma=0, total_iters=total_iters
        )
        scheduler = [
            PiecewiseConstantScheduler(
                optimizer, target_lr=target_lr, target_sigma=0, total_iters=total_iters
            )
            for optimizer in loptim
        ]
        train_loss, test_loss, test_ci = train(
            ocp,
            [optim_w, *loptim],
            [scheduler_w, *scheduler],
            f"L-{name}",
            epochs,
            train_size,
            test_size,
            train_batch,
            test_batch,
        )
        axs[i].plot(
            np.arange(len(test_loss)),
            test_loss,
            label=f"L-{name}",
        )
        axs[i].fill_between(
            np.arange(len(test_loss)),
            np.array(test_loss) - np.array(test_ci),
            np.array(test_loss) + np.array(test_ci),
            alpha=0.2,
        )

        # Layer Langevin algo
        for ll_rate in ll:
            ocp = DeepHedging(**model_config, N_euler=N)
            ocp.set_control(control_config, multiple_controls=True)
            optim_w = torch.optim.SGD(ocp.w.parameters(), lr=2e-3, momentum=0.9)
            scheduler_w = PiecewiseConstantScheduler(
                optim_w, target_lr=target_lr, target_sigma=0, total_iters=total_iters
            )
            lloptim = []
            scheduler = []
            for j, control in enumerate(ocp.control):
                if 100 * j / len(ocp.control) <= ll_rate:
                    lloptim.append(
                        langevin_optim(control.parameters(), lr=lr, sigma=sigma[i])
                    )
                else:
                    lloptim.append(optim(control.parameters(), lr=lr, **optim_kwargs))
                scheduler.append(
                    PiecewiseConstantScheduler(
                        lloptim[j],
                        target_lr=target_lr,
                        target_sigma=0,
                        total_iters=total_iters,
                    )
                )
            train_loss, test_loss, test_ci = train(
                ocp,
                [optim_w, *lloptim],
                [scheduler_w, *scheduler],
                f"LL-{name} {ll_rate}%",
                epochs,
                train_size,
                test_size,
                train_batch,
                test_batch,
            )
            axs[i].plot(
                np.arange(len(test_loss)),
                test_loss,
                label=f"LL-{name} {ll_rate}%",
            )
            axs[i].fill_between(
                np.arange(len(test_loss)),
                np.array(test_loss) - np.array(test_ci),
                np.array(test_loss) + np.array(test_ci),
                alpha=0.2,
            )

        if i == 0:
            axs[i].set_ylabel(r"$J(u_\theta)$")
        axs[i].set_xlabel("Epochs")
        if i == 2:
            legend = axs[i].legend(fancybox=True, edgecolor="k", loc=0)
            legend.get_frame().set_linewidth(0.5)

        axs[i].set_ylim((0.35, 1.2))

    plt.show()
    plt.close()

    return fig

In [ ]:
model_config = dict(
    T=1.0,
    dim=dim,
    ell=ell,
    a=torch.ones(dim).to(device),
    b=0.04 * torch.ones(dim).to(device),
    sigma=2.0 * torch.ones(dim).to(device),
    rho=-0.7 * torch.ones(dim).to(device),
    K=s0 * torch.ones(dim).to(device),
    T_COST=5e-4,
    S_0=s0 * torch.ones(dim).to(device),
    V_0=v0 * torch.ones(dim).to(device),
    device=device,
)

control_config = dict(
    input_dim=4 * dim,
    output_dim=2 * dim,
    hidden_dim=32,
    depth=2,
    activation=nn.ReLU(),
    out_transform=nn.ReLU(),
)

In [ ]:
adam_multiple_ctrl = test_diag_multiple_ctrls(
    model_config,
    [10, 20, 40],
    control_config,
    optim.Adam,
    LAdam,
    "Adam",
    2e-3,
    2e-4,
    [2e-4, 2e-4, 2e-4],
    [30, 90],
    200,
    180,
)

In [ ]:
adagrad_multiple_ctrl = test_diag_multiple_ctrls(
    model_config,
    [10, 20, 40],
    control_config,
    optim.Adagrad,
    LAdagrad,
    "Adagrad",
    7e-3,
    7e-4,
    [1e-5, 1e-5, 1e-5],
    [30, 90],
    200,
    180,
)

In [ ]:
adadelta_multiple_ctrl = test_diag_multiple_ctrls(
    model_config,
    [10, 20, 40],
    control_config,
    optim.Adadelta,
    LAdadelta,
    "Adadelta",
    5e-1,
    5e-2,
    [5e-3, 5e-3, 5e-3],
    [30, 90],
    200,
    180,
    rho=0.95,
)

In [ ]:
rmsprop_multiple_ctrl = test_diag_multiple_ctrls(
    model_config,
    [10, 20, 40],
    control_config,
    optim.RMSprop,
    LRMSprop,
    "RMSprop",
    2e-3,
    2e-4,
    [2e-3, 2e-3, 2e-3],
    [30, 90],
    200,
    180,
)

In [ ]:
sgd_multiple_ctrl = test_diag_multiple_ctrls(
    model_config,
    [10, 20, 40],
    control_config,
    optim.SGD,
    APLSGD,
    "SGD",
    5e-2,
    5e-3,
    [2e-3, 2e-3, 2e-3],
    [30, 90],
    200,
    180,
    momentum=0.9,
)

In [ ]:
if os.path.isdir("graphs/deephedging"):
    adam_multiple_ctrl.savefig("graphs/deephedging/adam_multiple_ctrl.pdf")
    adadelta_multiple_ctrl.savefig("graphs/deephedging/adadelta_multiple_ctrl.pdf")
    rmsprop_multiple_ctrl.savefig("graphs/deephedging/rmsprop_multiple_ctrl.pdf")
    sgd_multiple_ctrl.savefig("graphs/deephedging/sgd_multiple_ctrl.pdf")
    adagrad_multiple_ctrl.savefig("graphs/deephedging/adagrad_multiple_ctrl.pdf")

### Sample trajectory:

In [ ]:
alpha = 0.9
ell = lambda x: nn.functional.relu(x) / (1.0 - alpha)
dim = 1
s0 = 1.0
v0 = 0.1


model_config = dict(
    T=1.0,
    N_euler=50,
    dim=dim,
    ell=ell,
    a=torch.ones(dim),
    b=0.04 * torch.ones(dim),
    sigma=2.0 * torch.ones(dim),
    rho=-0.7 * torch.ones(dim),
    K=s0 * torch.ones(dim),
    T_COST=5e-4,
    S_0=s0 * torch.ones(dim),
    V_0=v0 * torch.ones(dim),
)

control_config = dict(
    input_dim=1 + 4 * dim,
    output_dim=2 * dim,
    hidden_dim=32,
    depth=2,
    activation=nn.ReLU(),
    out_transform=nn.ReLU(),
)
epochs = 100

In [ ]:
ocp = DeepHedging(**model_config)
ocp.set_control(control_config, multiple_controls=False)
optim_w = optim.SGD(ocp.w.parameters(), lr=2e-3, momentum=0.9)
adam_control = optim.Adam(ocp.control.parameters(), lr=2e-3)
adam = [optim_w, adam_control]

scheduler_w = PiecewiseConstantScheduler(
    optim_w, target_lr=2e-4, target_sigma=0, total_iters=80
)
scheduler_control = PiecewiseConstantScheduler(
    adam_control, target_lr=2e-4, target_sigma=0, total_iters=80
)
scheduler = [scheduler_w, scheduler_control]

In [ ]:
train_loss_adam, test_loss_adam, test_ci_adam = train(
    ocp,
    adam,
    scheduler,
    "Adam",
    epochs,
    train_size,
    test_size,
    train_batch,
    test_batch,
    False,
)

In [ ]:
S1, S2, V, u = ocp.sample_traj(1)

In [ ]:
fig = plt.figure(figsize=(10, 4))

t = model_config["T"] / model_config["N_euler"] * np.arange(model_config["N_euler"] + 1)
plt.plot(
    t,
    S1.flatten().detach().numpy(),
    label=r"$S_t^1$",
    marker="s",
    mec="k",
    ms=3,
    lw=1.5,
)
plt.plot(
    t, V.flatten().detach().numpy(), label=r"$V_t$", marker="s", mec="k", ms=3, lw=1.5
)
plt.plot(
    t,
    u[:, :, 0].flatten().detach().numpy(),
    label=r"$u_t^1$",
    marker="^",
    mec="k",
    ms=3,
    lw=1.5,
)
plt.plot(
    t,
    u[:, :, 1].flatten().detach().numpy(),
    label=r"$u_t^2$",
    marker="^",
    mec="k",
    ms=3,
    lw=1.5,
)

legend = plt.legend(fancybox=True, edgecolor="black", loc=0)
legend.get_frame().set_linewidth(0.5)
plt.title("Sample Trajectory")
plt.xlabel(r"$t$")
# fig.savefig("graphs/deephedging/Sample_traj_hedging_N50.pdf")
plt.show()
plt.close()

In [ ]:
alpha = 0.9
ell = lambda x: nn.functional.relu(x) / (1.0 - alpha)
dim = 5
s0 = 1.0
v0 = 0.1


model_config = dict(
    T=1.0,
    dim=dim,
    ell=ell,
    a=torch.ones(dim).to(device),
    b=0.04 * torch.ones(dim).to(device),
    sigma=2.0 * torch.ones(dim).to(device),
    rho=-0.7 * torch.ones(dim).to(device),
    K=s0 * torch.ones(dim).to(device),
    T_COST=5e-4,
    S_0=s0 * torch.ones(dim).to(device),
    V_0=v0 * torch.ones(dim).to(device),
)

control_config = dict(
    input_dim=1 + 4 * dim,
    output_dim=2 * dim,
    hidden_dim=32,
    depth=4,
    activation=nn.SiLU(),
    out_transform=nn.ReLU(),
)

In [ ]:
adam_single_ctrl = test_diag_single_ctrl(
    model_config,
    [30, 50, 100],
    control_config,
    torch.optim.Adam,
    LAdam,
    "Adam",
    2e-3,
    2e-4,
    [2e-5, 2e-5, 2e-5],
    100,
    80,
)